In [ ]:
# get insight into what tags are used in the documents and how often they occur

from nlppln.utils import get_files
import codecs
import re
import os
from collections import Counter

import sys

texts = {}
counters = {}

#fiqh = '/home/jvdzwaan/ownCloud/Shared/Bridging the gap/FiqhMaksim/Fiqh/Fiqh/'
#fiqh = '/home/jvdzwaan/ownCloud/Shared/Bridging the gap/Cleaned_data/Fiqh Cleaned 8-June/'
fiqh = '/home/jvdzwaan/code/fiqh_corpus/'
for in_file in get_files(fiqh):
    doc_id = os.path.basename(in_file)
    if in_file.endswith('.txt'):
        counters[doc_id] = Counter()
        with codecs.open(in_file, encoding='utf-8') as f:
            text = f.read()
            
        texts[doc_id] = text
        
        markers = re.findall('@(.+?)@', text)
        for marker in markers:
            
            counters[doc_id][marker] += 1
            # print markers that contain non-alphanumeric characters
            if not marker.isalnum():
                
                print '!!!', doc_id, marker, repr(marker)

        #print doc_id
        #for m, f in counters[doc_id].most_common():
        #    print m, f
        #print



In [ ]:
# Remove the non-printable characters from tags
for in_file in get_files(fiqh):
    doc_id = os.path.basename(in_file)
    if in_file.endswith('.txt'):
        with codecs.open(in_file, encoding='utf-8') as f:
            text = f.read()
    
        markers = re.findall('@(.+?)@', text)
        for marker in markers:
            
            counters[doc_id][marker] += 1
            # print markers that contain non-alphanumeric characters
            if not marker.isalnum():
                print '!!!', doc_id, marker, repr(marker)
                tag = ''.join(ch for ch in marker if ch.isalnum())
                text = text.replace(marker, tag)
         
        # Uncomment to save the changes to file
        #with codecs.open(in_file, 'w', encoding='utf-8') as f:
        #    f.write(text)

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(counters, orient='index')
df = df.fillna(0)
df

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
text = texts['Fiqh1.txt']

In [ ]:
# Remove metadata: remove all lines that start with #
text = text.replace('#.+\n', '')
text = re.sub(r'#.+\n', '', text)
print text

In [ ]:
for doc_id, text in texts.items():
    r1 = re.findall('PageV\d{2}P\d{3}', text)
    r2 = re.findall(' PageV\d{2}P\d{3} ', text)
    print doc_id, len(r1), len(r2)

In [ ]:
for doc_id, text in texts.items():
    r1 = re.findall('Milestone300', text)
    r2 = re.findall(' Milestone300 ', text)
    print doc_id, len(r1), len(r2)

In [ ]:
# check/clean metadata

metadata = {}
metadata_values = {}
md_fieldnames = {}

for doc_id, text in texts.items():
    metadata[doc_id] = Counter()
    md_fieldnames[doc_id] = Counter()
    r = re.findall(r'#META#(.+)\n', text)
    for m in r:
        splitted = m.split(u'::')
        if len(splitted)==2:
            name = splitted[0].strip()
            print name
            value = splitted[1].strip()
            metadata[doc_id][name] += 1
            if name[0].isdigit():
                md_fieldnames[doc_id]['numbers'] += 1
            else:
                md_fieldnames[doc_id]['text'] += 1
            if name not in metadata_values.keys():
                metadata_values[name] = Counter()
            metadata_values[name][value] += 1
        else:
            print doc_id, '\t\t\t\t', m

In [ ]:
# Header#End# without #META#
for doc_id, text in texts.items():
    metadata[doc_id] = Counter()
    r = re.findall(r'#META#Header#End#\n', text)
    if len(r) == 0:
        print doc_id


In [ ]:
numbers = pd.DataFrame.from_dict(md_fieldnames, orient='index')
numbers

In [ ]:
md = pd.DataFrame.from_dict(metadata, orient='index')
#md = md.fillna(0)
md

In [ ]:
mdv = pd.DataFrame.from_dict(metadata_values, orient='index')
#md = md.fillna(0)
mdv

In [ ]:
print mdv.columns